### this is my notebook for trying to figure out how to make medical history information into something useful

#### first we import libraries and data

In [1]:
import pandas as pd
import numpy as np

import os
import re
from collections import Counter

import re

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import sklearn.metrics
import xgboost as xgb

import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import sys
import random as rn

from IPython.display import clear_output


In [2]:
#use this cell when working online
path = '/floyd/home/ed-triage'
data_path = '/floyd/home/data'

In [2]:
#use this cell when working from home
path = '/Users/jjaskolkambp/Desktop/machine learning/my_projects/ed-triage'
data_path = '/Users/jjaskolkambp/Desktop/machine learning/my_projects/data/ED triage project/combo'

In [3]:
clin = pd.read_csv(data_path + '/complete_clean_combo_data.csv', index_col = 0,low_memory = False)

In [4]:
clin.shape

(165433, 121)

#### then we load up my helper functions

In [5]:

def show_confusion_matrix2(confusion_matrix, labels):
    LABELS = labels

    plt.figure(figsize=(3, 3))
    sns.heatmap(confusion_matrix, cmap = 'Blues', xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d", annot_kws={"size": 15});
    plt.title("Confusion matrix", fontsize=10)
    plt.ylabel('True label', fontsize=10)
    plt.xlabel('Predicted label', fontsize=10)
    plt.show()
    
def get_metrics(preds , predictions, show_confusion = True, print_output = True, return_values = False):
    
    y_test = targets.numpy()
    
    f1_w = sklearn.metrics.f1_score(y_test, predictions, average='weighted')
    f1 = sklearn.metrics.f1_score(y_test, predictions, average=None)
    acc = sklearn.metrics.accuracy_score(y_test, predictions)
    prec = sklearn.metrics.precision_score(y_test,predictions, average=None) 
    rec = sklearn.metrics.recall_score(y_test,predictions, average=None)
    auroc = sklearn.metrics.roc_auc_score(y_test,preds[:,1], average=None)
    confusion = sklearn.metrics.confusion_matrix(y_test, predictions)
    
    tp, fn, fp, tn = confusion[0,0], confusion[0,1], confusion[1,0], confusion[1,1]
    
    sens = tp/(tp + fn)
    spec = tn/(tn + fp)
    ppv = tp/(tp + fp)
    npv = tn/(tn + fn)
    
    
    if print_output:
        print ('Metrics Report:')
        print ('---------------')
        print ('weighted f1: ', f1_w)
        print ('AUROC:       ',auroc)
        print ('accuracy:    ', acc)
        print ('precision:   ', prec)
        print ('recall:      ', rec)
        print ('sensitivity: ', sens)
        print ('specificity: ', spec)
        print ('PPV:         ', ppv)
        print ('NPV:         ', npv)
        
    if show_confusion:
        show_confusion_matrix2(confusion, labels = data.classes)
     
    if return_values == True:
        return f1_w, auroc, acc, sens, ppv



#### then we clean up the data in the medical history column

In [6]:
history = list(clin['MedicalHistory'].str[16:].astype('str'))

In [381]:
#this is to clean the text in the medical history column (a little bit)
def fixhx(s):
    s =s.replace('<LT><LF>','')
    s = s.replace('<LT>LF>','')
    s = s.replace('<LF><LF>','')
    s =s.replace('Pt.', 'patient')
    s =s.replace('Pt', 'patient')
    s = s.lower()
    s = s.replace('pt.', 'patient')
    s = s.replace('y/o', 'year old')
    s = s.replace('c/o', 'complains of')
    s = s.replace('h/o', 'history of')
    s = s.replace('sob', 'shortness of breath')
    s = s.replace('c/p', 'chest pain')
    s = s.replace('hrs', 'hours')
    s = s.replace('hx', 'history')
    s = s.replace('n/v', 'nausea and/or vomiting')
    s = s.replace('a/e', 'air entry')
    s = s.replace('a/o', 'alert and oriented')
    s = s.replace('a&o', 'alert and oriented')
    s = s.replace('d/c', 'discharge')
    s = s.replace('u/s', 'ultrasound')
    s = s.replace('yrs', 'years')
    s = s.replace('lmp', 'last menstrual period')
    s = s.replace('w/', 'with')
    s = s.replace('@', 'at ')
    s = s.replace('b/c', 'because')
    s = re.sub(r'\br\s*\/\s*o','rule out',s)
    s = re.sub(r'\bc\s*\/\s*o','complains of',s)
    s = re.sub(r'\bpt.','patient ',s)
    s = re.sub(r'\blt.','left ',s)
    s = re.sub(r'\brt.','right ',s)
    s = re.sub(r'\bmd.','doctor ',s)
    s = re.sub(r'\bra\b','room air ',s)
    s = re.sub(r'\bbp.','blood pressure ',s)
    s = re.sub(r'\bcp.','chest pain ',s)
    s = re.sub(r'\bfd.','family doctor ',s)
    s = s.replace('abx', 'antibiotics')
    s = s.replace(' htn ', ' hypertension ')
    s = re.sub(r'\btyl\b', 'tylenol', s)
    s = re.sub(r'\bprn\b', 'as needed ',s)
    s = re.sub(r'\bvag\b', 'vaginal ',s)
    s = re.sub(r'\bpo\b', 'by mouth ',s)
    s = re.sub(r'\bllq\b', 'left lower quadrant',s)
    s = re.sub(r'\brlq\b', 'right lower quadrant  ',s)
    s = re.sub(r'\bluq\b', 'left upper quadrant',s)
    s = re.sub(r'\bruq\b', 'right upper quadrant  ',s)
    s = re.sub(r'\buti\b', 'urinary tract infection ',s)
    s = re.sub(r'\brd', 'right digit number',s)
    s = re.sub(r'\bld', 'left digit number',s)
    s = re.sub(r'\br\b', 'right ',s)
    s = re.sub(r'\bl\b', 'left ',s)
    s = re.sub(r'\bed\b', 'emergency department ',s)
    s = re.sub(r'\ber\b', 'emergency room ',s)
    s = re.sub(r'\bbm\b', 'bowel movement ',s)
    s = re.sub('(?<=[a-zA-Z])-(?=[a-zA-Z])', ' ', s)
    s = re.sub(r'\bwic\b', 'walk in clinic ',s)
    s = re.sub(r'\bems\b', 'emergency medical services ',s)
    s = re.sub(r'\bmvc\b', 'motor vehicle collision  ',s)
    s = re.sub(r'\bgp\b', 'family doctor  ',s)
    s = re.sub(r'\bloc\b', 'loss of consciousness ',s)
    s = re.sub(r'\bsts\b', 'states  ',s)
    s = re.sub(r'\bmvc', 'motor vehicle collision ',s)
    s = re.sub(r'\bnsr\b', 'normal sinus rhythm ',s)
    s = s.replace("--", "-")
    s = s.replace('-', ' ')
    s = re.sub('and/or', 'or', s)
    s = re.sub(r'\bneg', 'negative', s)
    s = re.sub('s/b', 'sent by', s)
    s = re.sub(r'nausea\s*\/\s*vomiting', 'nausea or vomiting', s)
    s = re.sub(';',',',s)
    s = re.sub('aaox3', 'alert and oriented', s)
    s = re.sub(r'csm', ' circulation, sensation and movement ',s)
    s = re.sub(r'sz', ' seizure ',s)
    s = re.sub('hgb', 'hemoglobin', s)
    s = re.sub('\basa\b', 'aspirin', s)
    s = re.sub('\butd\b', 'up to date', s)
    s = re.sub('\brx\b', 'treatment', s)
    s = re.sub('\bnpo\b', 'nothing by mouth', s)
    s = re.sub('d/c', 'discharge', s)
    s = re.sub('rxn', 'reaction', s)
    #s = re.sub(r"(\b[a-z]\b)\s([a-z])", r"\1-\2", s)  #added this in to try and deal with c section, a fib, h pylori
    
    
    
    punctuation_spacer = str.maketrans({key: f"{key} " for key in ".,"})
    s = s.translate(punctuation_spacer)
    
    s = re.sub(r'\s+,',',', s)
    s = re.sub(r'\s+\.','.', s)

    
    
    #remove extra white spaces
    s = re.sub(r'\s+', ' ', s)

    #note that what's missing here is removal of punctuation
    return s

In [382]:
replacements = (',','<LT>LF>', '.', ';', '/')
drop_list = ('"', '?', 'had', 'has', 'have', 'left', 'right', 'sided','weeks', 'history of', 'h/o','and')   #added all but first 2 options may 30, 2020
split_hist = []
for item in history:
    for r in replacements:
        item = item.replace(r, '::')
        
    item = item.split('::')
    #item = [re.sub(r"(\b[a-z]\b)\s([a-z])", r"\1-\2", s) for s in item]  #to handle a fib, c section, h pylori
    item = [re.sub(r'\(.*?\)\ *', '', s) for s in item]  #remove punctuation
    #item = [re.sub(r'\b\d+\b', '', s) for s in item]  #remove numbers
    item = [re.sub(r'%', '', s) for s in item]  #remove percent sign
    item = [re.sub(r'\b[0-9]\w+''', '', s) for s in item]  #remove words that start with numbers
    item = [re.sub(r'^[^\w#]*', '',s) for s in item]    #remove leading symbols except fracture
    item = [re.sub(r'^-\s*', '',s) for s in item] #remove leading dashes
    item = [fixhx(s) for s in item]   #run the cleaning function above to get rid of some of the abbreviation
    
    
    for d in drop_list:
        item = [x.replace(d,'') for x in item]   #get rid of the words from the blacklist above
    
    item = [x.strip().lower() for x in item if x != 'nan']   #get rid of white spaces
    item = [x for x in item if len(x) > 2]
    item = list(filter(None, item))
    
    split_hist.append(item)
    
    
 #split hist is a list of lists, where each entry is the list of past medical history conditions for each patient

In [383]:
#split hist is a list of lists, where each entry is the list of past medical history conditions for each patient
split_hist[:30]

[['no significant medical history'],
 ['no significant medical history'],
 ['no significant medical history'],
 ['dialysis haemo', 'iddm', 'crf', 'heart problem', 'high cholesterol'],
 ['fibromyalgia'],
 ['enlarged liver'],
 ['anxiety', 'autistic', 'adhd', 'depression', 'asthma', 'celiac'],
 ['no significant medical history'],
 ['hypothyroid'],
 ['anxiety'],
 ['no significant medical history'],
 ['hypertension'],
 ['arthritis', 'copd'],
 ['itp', 'wiscott aldridge syndrome', 'eczema'],
 ['suicidal attempts', 'depression'],
 ['high cholesterol',
  'hypertension',
  'niddm',
  'cabg',
  'knee surgery years ago'],
 ['sepsis',
  'urinary tract infection',
  'ca lungs',
  'ca bronchus',
  'hypothyroid',
  'glaucoma cataract',
  'cataract repair',
  'kidney disease',
  'anemia'],
 ['alcoholism'],
 ['gerd', 'high cholesterol'],
 ['hypertension',
  'tia',
  'osteoporosis',
  'diverticulitis',
  'high cholesterol',
  'anxiety',
  'depression',
  'aortic stenosis'],
 ['chf', 'hypertension', 'nidd

In [396]:
#my hideous way of limiting each diagnosis to 2 words long
split_hist2 = []
for item in split_hist:
    item2 = []
    for dx in item:
        if dx != 'no significant medical history':
            #dx = re.sub(r"(\b[a-z]\b)\s([a-z])", r"\1-\2", dx)
            new_dx = ' '.join(dx.split()[:2])
            item2.append(new_dx)
        else:
            item2.append(dx)
    split_hist2.append(item2)

In [385]:
#I've processed it so each diagnosis is no more than 2 words long
split_hist2[:10]

[['no significant medical history'],
 ['no significant medical history'],
 ['no significant medical history'],
 ['dialysis haemo', 'iddm', 'crf', 'heart problem', 'high cholesterol'],
 ['fibromyalgia'],
 ['enlarged liver'],
 ['anxiety', 'autistic', 'adhd', 'depression', 'asthma', 'celiac'],
 ['no significant medical history'],
 ['hypothyroid'],
 ['anxiety']]

### exploring different diagnoses

In [386]:
len(split_hist2)

165433

In [397]:
diagnoses = Counter()
for hx in split_hist2:   #note that this is done with my split modified split hist
    for item in hx:
        diagnoses[item] += 1

In [388]:
#there were 23721 different medical histories used in the dataset!!!
#now down to 15852
len(diagnoses)

15947

In [389]:
diagnoses.most_common(100)

[('no significant medical history', 73713),
 ('hypertension', 21678),
 ('high cholesterol', 19789),
 ('niddm', 12418),
 ('htn', 7804),
 ('asthma', 6985),
 ('depression', 6437),
 ('high bp', 6406),
 ('thyroid', 4958),
 ('anxiety', 4913),
 ('hypothyroid', 3987),
 ('iddm', 3812),
 ('gerd', 3427),
 ('diabetes', 3263),
 ('acid reflux', 2423),
 ('arthritis', 2230),
 ('anemia', 1710),
 ('copd', 1523),
 ('high chol', 1297),
 ('chf', 1180),
 ('schizophrenia', 1155),
 ('gout', 1128),
 ('seizure', 1064),
 ('cva', 1064),
 ('kidney stones', 1056),
 ('enlarged prostate', 1021),
 ('atrial fibrillation', 954),
 ('cholesterol', 951),
 ('pacemaker', 939),
 ('dementia', 939),
 ('migraines', 875),
 ('adhd', 797),
 ('osteoporosis', 789),
 ('chronic back', 740),
 ('afib', 733),
 ('bipolar', 730),
 ('immunizations up', 688),
 ('smoker', 678),
 ('urinary tract', 633),
 ('cad', 627),
 ('fibromyalgia', 602),
 ('substance misuse', 597),
 ('breast cancer', 584),
 ('pneumonia', 567),
 ('chol', 546),
 ('defib', 544

In [390]:
sorted(diagnoses.keys())[500:570]

['acute angina',
 'acute asceding',
 'acute cholecystitis',
 'acute diverticulitis',
 'acute exacerbations',
 'acute gastritis',
 'acute hepatitis',
 'acute immune',
 'acute ischemic',
 'acute kidney',
 'acute lymphoblastic',
 'acute macular',
 'acute myeloid',
 'acute myelomonocytic',
 'acute mylo',
 'acute myoblastic',
 'acute or',
 'acute osteomyelitis',
 'acute pancreatitis',
 'acute pericarditis',
 'acute psychosis',
 'acute renal',
 'acute rhinosinisitis',
 'acute rt',
 'acyclovir',
 'aczema',
 'ad )',
 'adad',
 'adalat',
 'adams oliver',
 'adanoymiosis',
 'add',
 'add depresssion',
 'addh',
 'addhd',
 'addiction',
 'addiction to',
 'addictions',
 'addictive sexual',
 'addision',
 'addison disease',
 "addison's-disease",
 'addisons',
 'addisons disease',
 "addisson's-disease",
 'additction to',
 'adem meningitis',
 'aden',
 'aden carcinoma',
 'adenectomy',
 'adenocarcinoma',
 'adenocarcinoma lung',
 'adenocarcinoma of',
 'adenocarcinoma prostate',
 'adenoid',
 'adenoid ca',
 'ade

In [398]:
sorted(diagnoses.keys())[500:570]

['actonel',
 'acutane',
 'acute',
 'acute angina',
 'acute asceding',
 'acute cholecystitis',
 'acute diverticulitis',
 'acute exacerbations',
 'acute gastritis',
 'acute hepatitis',
 'acute immune',
 'acute ischemic',
 'acute kidney',
 'acute lymphoblastic',
 'acute macular',
 'acute myeloid',
 'acute myelomonocytic',
 'acute mylo',
 'acute myoblastic',
 'acute or',
 'acute osteomyelitis',
 'acute pancreatitis',
 'acute pericarditis',
 'acute psychosis',
 'acute renal',
 'acute rhinosinisitis',
 'acute rt',
 'acyclovir',
 'aczema',
 'ad )',
 'adad',
 'adalat',
 'adams oliver',
 'adanoymiosis',
 'add',
 'add depresssion',
 'addh',
 'addhd',
 'addiction',
 'addiction to',
 'addictions',
 'addictive sexual',
 'addision',
 'addison disease',
 "addison's-disease",
 'addisons',
 'addisons disease',
 "addisson's-disease",
 'additction to',
 'adem meningitis',
 'aden',
 'aden carcinoma',
 'adenectomy',
 'adenocarcinoma',
 'adenocarcinoma lung',
 'adenocarcinoma of',
 'adenocarcinoma prostate'

In [393]:
poss_words('addison\'s disease')

["addison's disease", 'addison disease', 'addisons disease']

### this is getting much better.  gonne now do some work to make all the aaa's, c-sections, etc. collapse to a single diagnosis

In [349]:
#this function takes in a counter with word frequencies of similar words
#it returns the most common spelling as the "best spelling"

def get_best_spelling(word, dict_name):
    #print (word)
    max_value = -10
    best_spelling = 0
    for k,v in dict_name.items():
        if v > max_value:
            max_value = v
            best_spelling = k
    
    if max_value == 1:
        best_spelling = word
        
    return best_spelling 

In [350]:
#this is stolen from peter norvigs spellchecker
import re
from collections import Counter

#def words(text): return re.findall(r'\w+', text.lower())

WORDS = diagnoses

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

#### now i need to find a way to generate the frequency dict/counter to use to get the best spelling

In [351]:
#gonna start using the functions from peter norvig
#arbitrarily using one of my c-words:  'cataracts'

def poss_words(word):
    return list(set(list(known(edits1(word))) +  list(known(edits2(word))) + [word]))

In [352]:
print (poss_words('c-section'))
print (poss_words('c section'))
print (poss_words('cataract'))
print (poss_words('crohns'))
print(poss_words('c-diff'))

['c-sction', 'c-cesection', 'csection', 'disection', 'c-sections', 'csections', 'c-section', 'resection']
['c section', 'csection', 'disection', 'csections', 'resection']
['caatracts', 'cataract', 'cateract', 'catarract', 'cataracts', 'catarac', 'catarct', 'catract', 'catracts']
['cons', 'crohns', 'chrohns', 'crohn', 'chrons', 'croups', 'cronhs']
['diff', 'c-dif', 'c-diff', 'cdiffe', 'cdif', 'cdiff']


this function takes the diagnoses counter and generates a dictionary of possible spellings for each first word > 5 letters, beginning with the chosen letter

In [356]:
def first_letter_counter(letter, diagnosis_counter = diagnoses, word_length=5):
    collected_words = [word for word in sorted(diagnosis_counter.keys()) if word[0] == letter]
    
    let_counter = Counter(collected_words)
    
    first_words = []
    for word in let_counter.keys():
        first_word =  word.split()[0]
        first_words.append(first_word) 
        
    fw_counter = Counter(first_words)
    
    spell_dict = {}
    for first_word in first_words:
        if len(first_word) > word_length:
            poss_spells = poss_words(first_word)
            word_count = dict()
            for spell in poss_spells:
                word_count[spell] = fw_counter[spell]
            spell_dict[first_word] = word_count
        
    return collected_words, spell_dict

In [357]:
#this should have the same results as the cell by cell implementation with the letter c
%time c_collected_words, c_spell_dict = first_letter_counter('c', diagnoses, 4)

CPU times: user 2min 26s, sys: 606 ms, total: 2min 27s
Wall time: 2min 30s


In [358]:
for k,v in c_spell_dict.items():
    print (k,':')
    print (v)
    print (get_best_spelling(k,v))
    print ()

c'd-yesterday :
{"c'd-yesterday": 1}
c'd-yesterday

c-after :
{'c-after': 1}
c-after

c-anca :
{'c-anca': 1}
c-anca

c-bound :
{'c-bound': 1}
c-bound

c-cesection :
{'c-cesection': 1, 'c-section': 37}
c-section

c-cestionsx :
{'c-cestionsx': 1}
c-cestionsx

c-complete :
{'incomplete': 0, 'icomplete': 0, 'c-complete': 1}
c-complete

c-dif :
{'c-dif': 1, 'c-diff': 7, 'd-def': 0, 'cdif': 1, 'cdiff': 3}
c-diff

c-diff :
{'diff': 0, 'c-dif': 1, 'c-diff': 7, 'cdiffe': 1, 'cdif': 1, 'cdiff': 3}
c-diff

c-fracture :
{'fracture': 0, 'c-fracture': 1, 'h-fracture': 0, 't-fracture': 0}
c-fracture

c-joint :
{'c-joint': 1}
c-joint

c-last :
{'c-last': 1}
c-last

c-lip :
{'c-dif': 1, 'c-pap': 2, 'c-lip': 1, 'coli': 1, 'colic': 1}
c-pap

c-palate :
{'c-palate': 3}
c-palate

c-pap :
{'c-pap': 2, 'bipap': 0}
c-pap

c-sction :
{'c-section': 37, 'csection': 2, 'c-sction': 1, 'c-sections': 1}
c-section

c-sec :
{'c-sec': 1, 'losec': 0}
c-sec

c-sect :
{'c-sect': 3, 'c-sec': 1}
c-sect

c-section :
{'c-scti

collapsed

collar :
{'collar': 2}
collar

collasped :
{'collasped': 1}
collasped

colle :
{'colon': 27, 'cll': 2, 'colds': 1, 'colle': 1, 'choll': 1, 'coli': 1, 'colic': 1, 'colace': 1, 'coil': 1, 'pollen': 0, 'colo': 2, 'chole': 1, 'cold': 5, 'coils': 1}
colon

collitis :
{'colititis': 1, 'colittis': 1, 'cellutis': 1, 'celulitis': 1, 'cholitis': 1, 'colitis': 2, 'clolitis': 1, 'cholelitis': 1, 'collitis': 1, 'celluitis': 1, 'collititis': 1, 'coitis': 1}
colitis

collititis :
{'cellutitis': 1, 'cellilitis': 1, 'colititis': 1, 'colittis': 1, 'collitis': 1, 'collititis': 1}
collititis

colocn :
{'colocn': 1, 'colon': 27, 'colic': 1, 'colace': 1, 'colo': 2}
colon

colon :
{'chron': 1, 'olan': 0, 'colon': 27, 'colds': 1, 'coli': 1, 'colic': 1, 'colo': 2, 'cold': 5}
colon

colonectomy :
{'colonectomy': 1, 'colectomy': 2}
colectomy

colonic :
{'colon': 27, 'chronic': 149, 'colic': 1, 'colonic': 4}
chronic

colonnscopy :
{'colonosxopy': 1, 'colonnscopy': 1, 'colonscopy': 7, 'colonoscopy': 8, 

In [359]:
#c_spell_dict should be the same as c_spell_counter above
#c_collected_words should be the same as c_words above
len(c_spell_dict), len (c_collected_words), len(set(c_collected_words))

(561, 1599, 1599)

looking good.  let's try with another letter for fun

In [361]:
%time h_diagnoses, h_spell_dict = first_letter_counter('h', diagnoses, 5)

KeyboardInterrupt: 

In [362]:
#h_spell_dict should be shorter than h_diagnoses becuase it will only contain the first word of each diagnosis
len(h_diagnoses), len(set(h_diagnoses)), len(h_spell_dict)

(1113, 975, 394)

In [363]:
for k,v in h_spell_dict.items():
    print (k,':')
    print (v)
    print (get_best_spelling(k,v))
    print ()

haalluciations :
{'haalluciations': 1, 'hallucinations': 3}
hallucinations

haemangioma :
{'hemangioma': 3, 'haemangioma': 1}
hemangioma

haemhorroids :
{'hemorroids': 1, 'haemhorroids': 1, 'haemorrhoids': 1, 'hemmorroids': 1}
haemhorroids

haemodialysis :
{'hemodilaysis': 1, 'hemodailysis': 1, 'haemodialysis': 1, 'hemodialysis': 9}
hemodialysis

haemophilia :
{'hemophilia': 3, 'hemophillia': 1, 'haemophilia': 1}
hemophilia

haemorrhoid :
{'hemorrhoids': 5, 'hemorrhoid': 3, 'hemmorrhoid': 1, 'hemorroid': 2, 'hemorhoid': 1, 'haemorrhoid': 2, 'haemorrhoids': 1}
hemorrhoids

haemorrhoids :
{'hemorhroids': 1, 'hemorrhords': 1, 'hemorrhoids': 5, 'hemorrhoid': 3, 'hemorroids': 1, 'hemmorrhoids': 1, 'hemerrhoids': 1, 'hemrrhoids': 1, 'hemorrihoids': 1, 'hamrrhoids': 1, 'haemhorroids': 1, 'hemorhoids': 1, 'hemorrhiods': 1, 'haemorrhoid': 2, 'hemoorhoids': 1, 'haemorrhoids': 1}
hemorrhoids

hairline :
{'hairline': 1}
hairline

haliperdol :
{'haliperdol': 1}
haliperdol

hallucination :
{'halluci

hypertension

hypthyrodism :
{'hypothyroism': 1, 'hyothyroidism': 1, 'hypothryodism': 1, 'hypthyroidism': 1, 'hypthyrodism': 1, 'hypothyridism': 1, 'hypothyroidism': 1, 'hypothyroidsm': 1, 'hyperthyrodism': 1, 'hypothyrodism': 1}
hypthyrodism

hypthyroid :
{'hypothyrid': 1, 'hypthyroid': 1, 'hypothyroid': 3, 'hypothroid': 1, 'hypothryoid': 1, 'hypotyhroid': 1, 'hyothyroid': 1, 'hyphothyroid': 1, 'hypothryroid': 1, 'hypithyroid': 1, 'hypohyroid': 1, 'yhpothyroid': 0, 'hythyroid': 1, 'hyperthyroid': 1}
hypothyroid

hypthyroidism :
{'hypothyroidsim': 1, 'hyothyroidism': 1, 'hypothyroidis': 1, 'hypthyroidism': 1, 'hopythyroidism': 1, 'hypthyrodism': 1, 'hyptothyroidism': 1, 'hypothyridism': 1, 'hyperthyroidism': 1, 'hypothyroidism': 1, 'hypothyroidisn': 1, 'hypothyroidsm': 1, 'hypothroidism': 1, 'hypothyrodism': 1, 'hypothyroidim': 1}
hypthyroidism

hyptothyroidism :
{'hypothyroidsim': 1, 'hyothyroidism': 1, 'hypothyroidis': 1, 'hypthyroidism': 1, 'hyptothyroidism': 1, 'hypothyridism': 1, 

In [364]:
%time a_diagnoses, a_spell_dict = first_letter_counter('a', diagnoses, 5)

CPU times: user 2min 13s, sys: 802 ms, total: 2min 13s
Wall time: 2min 19s


In [365]:
len(a_diagnoses), len(set(a_diagnoses)), len(a_spell_dict)

(1394, 1394, 533)

In [366]:
for k,v in a_spell_dict.items():
    print (k,':')
    print (v)
    print (get_best_spelling(k,v))
    print ()

a-cyst :
{'a-cyst': 1, 'cyst': 0}
a-cyst

a-cystoscopy :
{'cystoscopy': 0, 'a-cystoscopy': 1}
a-cystoscopy

a-fibrillation :
{'afibrillation': 1, 'a-fibrillation': 1}
a-fibrillation

a-flutter :
{'aflutter': 1, 'flutter': 0, 'a-flutter': 1}
a-flutter

a-leaky :
{'a-leaky': 1}
a-leaky

a-lumpectomy :
{'a-lumpectomy': 1, 'lumpectomy': 0}
a-lumpectomy

a-osteoarthritis :
{'osteoarthritis': 0, 'a-osteoarthritis': 1}
a-osteoarthritis

a-shunt :
{'a-shunt': 1, 'shunt': 0}
a-shunt

a-surgery :
{'h-surgery': 0, 'a-surgery': 1, 'surgery': 0}
a-surgery

a-tube :
{'g-tube': 0, 'gtube': 0, 'jtube': 0, 'a-tube': 1}
a-tube

a-twin :
{'a-twin': 1}
a-twin

aarthritis :
{'arthritis': 14, 'arthrirtis': 1, 'arthirtis': 1, 'arthristis': 2, 'arthrits': 1, 'arhtritis': 1, 'arrthritis': 1, 'arthtritis': 1, 'arthitis': 2, 'arthrities': 1, 'arthrtis': 1, 'athritis': 1, 'arhthritis': 1, 'artritis': 1, 'arthrritis': 1, 'aathritis': 1, 'aarthritis': 1, 'arthtitis': 1, 'arthiritis': 1, 'arthrittis': 1}
arthritis



antibiotic :
{'antibiotics': 5, 'antibiotic': 2, 'antiobiotics': 1}
antibiotics

antibiotics :
{'antibiotics': 5, 'antibiotic': 2, 'antiobiotics': 1}
antibiotics

antibodies :
{'antibodies': 1}
antibodies

anticoagulants :
{'anticoagulants': 1}
anticoagulants

antidepressant :
{'antidepressant': 1, 'antidepressants': 1}
antidepressant

antidepressants :
{'antidepressant': 1, 'antidepressants': 1}
antidepressants

antinausea :
{'antinausea': 1}
antinausea

antiobiotics :
{'antibiotics': 5, 'antibiotic': 2, 'antiobiotics': 1}
antibiotics

antiplatelets :
{'antiplatelets': 1}
antiplatelets

antipsychotic :
{'antipsychotic': 1}
antipsychotic

antisocial :
{'antisocial': 3}
antisocial

antiviral :
{'antiviral': 1}
antiviral

anuerysm :
{'aneursym': 3, 'aneurism': 3, 'aneurysm': 10, 'anurysym': 1, 'anyersm': 1, 'anuerysm': 2}
aneurysm

anueyearsm :
{'anueyearsm': 1}
anueyearsm

anuresim :
{'anuresim': 1, 'anurysym': 1}
anuresim

anurysym :
{'aneursym': 3, 'aneurysm': 10, 'anurysym': 1, 'anur

avascular :
{'avascular': 1}
avascular

averbal :
{'averbal': 1}
averbal

avscular :
{'avscular': 1}
avscular

awaiting :
{'awaiting': 6}
awaiting

axiety :
{'anxietyw': 1, 'anxiety': 13, 'anaxiety': 1, 'ansiety': 1, 'anxeity': 2, 'anxiey': 2, 'anixiety': 1, 'anciety': 1, 'anxitey': 1, 'anxety': 1, 'naxiety': 0, 'anixety': 1, 'anxietry': 1, 'aniety': 1, 'anxiet': 1, 'asnxiety': 1, 'anxirty': 1, 'axiety': 1, 'anxierty': 1, 'anxity': 1, 'amxiety': 1}
anxiety

azathioprine :
{'azathioprine': 1}
azathioprine

azithromycin :
{'azithromycin': 1}
azithromycin



### now gonna work on correcting the diagnoses

In [223]:
#gonna copy the first stage of h diagnoses and overwrite the new words with correct spellings
#using h arbitrarily here...
h_diagnoses_old = h_diagnoses.copy()

In [227]:
#this will go through a list of diagnoses, pull out the first word, see if the first word is in the
#dictionary fo alternate spellings for a word and replace it with the best spelling

for i in range(len(h_diagnoses)):
    word_list = h_diagnoses[i].split()
    
    if word_list[0] in h_spell_dict.keys():
        best_word = get_best_spelling(word_list[0], h_spell_dict[word_list[0]])
        
        if word_list[0] != best_word: 
            word_list[0] = best_word
    
    h_diagnoses[i] = ' '.join(word_list)

   

In [199]:
print(len(h_diagnoses))
print (len(set(h_diagnoses)))

1113
982


In [200]:
#this is the same length as the dict above but the first word has been corrected in many cases
h_diagnoses[:30]

['h amputee',
 'h burn',
 'h foot',
 'h fracture',
 'h infection',
 'h injury',
 'h numbness',
 'h pain',
 'h piloro',
 'h plyori',
 'h plyorie',
 'h pylori',
 'h pyloric',
 'h pylorie',
 'h surgery',
 'h sx',
 'h years',
 'h1n1',
 'ha d',
 'hallucinations',
 'hemangioma',
 'haemhorroids',
 'hemodialysis',
 'hemophilia',
 'hemorrhoids',
 'hemorrhoids problem',
 'hemorrhoids',
 'hair loss',
 'hair procedure',
 'hairline fracture']

In [228]:
#but it hasnt shortened that much
len(Counter(h_diagnoses)), len(Counter(h_diagnoses_old))

(975, 1113)

running the function again to see if it changes the corrections at all...

In [229]:
h_counter2 = (Counter(h_diagnoses))
print (len(h_counter2))
%time h_diagnoses2, h_spell_dict2 = first_letter_counter('h', h_counter2, 5)

975
CPU times: user 1min 17s, sys: 286 ms, total: 1min 17s
Wall time: 1min 19s


In [230]:
#and it doesnt
len(Counter(h_diagnoses2))

975

#### now we are going to see if we can correct whole diagnoses

In [235]:
h_counter2.most_common(50)

[('hemorrhoids', 29),
 ('hypertension', 17),
 ('hypothyroid', 17),
 ('hysterectomy', 13),
 ('hydrocephalus', 9),
 ('hernia', 6),
 ('hypoglycemia', 5),
 ('hemodialysis', 4),
 ('hemophilia', 4),
 ('hepatitis b', 4),
 ('himotos disease', 4),
 ('hydronephrosis', 4),
 ('hyperemesis', 4),
 ('hallucinations', 3),
 ('hemangioma', 3),
 ('himotos', 3),
 ('hypokalemia', 3),
 ('hysterectomy years', 3),
 ('heart attack', 2),
 ('head', 2),
 ('headache', 2),
 ('heart stents', 2),
 ('heart valve', 2),
 ('hemodialysis m', 2),
 ('hemorrhoids sx', 2),
 ('hemorrhoidectomy', 2),
 ('hepatitis', 2),
 ('hepatic b', 2),
 ('hernia repair', 2),
 ('hernia use', 2),
 ('hernia user', 2),
 ('hidradenitis suppurativa', 2),
 ('huntingtons chorea', 2),
 ('hyper', 2),
 ('hyper lipidemia', 2),
 ('hyperacidity', 2),
 ('hysterectomy last', 2),
 ('hysterectomy in', 2),
 ('hysterectomy on', 2),
 ('h amputee', 1),
 ('h burn', 1),
 ('h foot', 1),
 ('h fracture', 1),
 ('h infection', 1),
 ('h injury', 1),
 ('h numbness', 1),
 (

In [237]:
poss_words('h pylori')

['h pylori',
 'h pyloric',
 'h piloro',
 'h pylorie',
 'h plyori',
 'h plyorie',
 'pylori',
 'hpylori',
 'hp pyloric',
 'hel pylori']

In [232]:
poss_words('hysterectomy')

['hystrectomy',
 'hysterrectomy',
 'histerectomy',
 'hyperectomy',
 'hysterectomuy',
 'hystprectomy',
 'hysterectomy',
 'hestorectomy',
 'hyserectomy',
 'hyterectmomy',
 'hyterectomy',
 'hysterctomy']

In [233]:
poss_words('heart attack')

['hert attack',
 'hear attack',
 'heart attack',
 'heart attacked',
 'heartattack',
 'heart attacks',
 'heaart attack']

very promising as it finds lots of similar diagnoses

In [177]:
def whole_diagnosis_counter(diagnosis_counter = diagnoses):
    
    spell_dict = {}
    for diagnosis in list(diagnosis_counter.keys()):
        #print (diagnosis)
        poss_spells = poss_words(diagnosis)
        #print (poss_spells)
        word_count = dict()
        for spell in poss_spells:
            #print (spell)
            word_count[spell] = diagnosis_counter[spell]
        spell_dict[diagnosis] = word_count
        
    return spell_dict

In [178]:
list(h_counter.keys())[:50]

['h amputee',
 'h burn',
 'h foot',
 'h fracture',
 'h infection',
 'h injury',
 'h numbness',
 'h pain',
 'h piloro',
 'h plyori',
 'h plyorie',
 'h pylori',
 'h pyloric',
 'h pylorie',
 'h surgery',
 'h sx',
 'h years',
 'h1n1',
 'ha d',
 'hallucinations',
 'hemangioma',
 'hemorroids',
 'hemodialysis',
 'hemophilia',
 'hemorrhoids',
 'hemorrhoids problem',
 'hair loss',
 'hair procedure',
 'hairline fracture',
 'half of',
 'haliperdol',
 'hallucinations nyd',
 'hallucinations post',
 'hamilton hall',
 'hamstring strain',
 'hamstring tare',
 'hamstring tear',
 'harada disease',
 'hard of',
 'hard on',
 'haringtons',
 'hartmann reversal',
 'hay fever',
 'hb low',
 'hbd',
 'hbh',
 'hbv',
 'hcl',
 'hctz',
 'hd tuethusat']

In [179]:
%time spell_dict = whole_diagnosis_counter(h_counter)

CPU times: user 3min 57s, sys: 873 ms, total: 3min 57s
Wall time: 4min 5s


In [184]:
for k,v in spell_dict.items():
    opt = get_best_spelling(k,v)
    if v != opt:  #only showing the ones where there could be a change in spelling
        print (k,':')
        print (v)
        print (get_best_spelling(k,v))
        print ()

h amputee :
{'h amputee': 1, 'amputee': 0}
h amputee

h burn :
{'h burn': 1, 'burn': 0}
h burn

h foot :
{'h foot': 1, 'foot': 0}
h foot

h fracture :
{'t fracture': 0, 'c fracture': 0, 'h fracture': 1, 'fracture': 0, 'hip fracture': 1}
h fracture

h infection :
{'infection': 0, 'h infection': 1}
h infection

h injury :
{'injury': 0, 'no injury': 0, 'hip injury': 1, 'h injury': 1}
h injury

h numbness :
{'h numbness': 1}
h numbness

h pain :
{'hip pain': 1, 'h pain': 1, 'pain': 0}
h pain

h piloro :
{'h pylori': 1, 'h piloro': 1}
h piloro

h plyori :
{'h pylori': 1, 'h pyloric': 1, 'h pylorie': 1, 'h plyori': 1, 'h plyorie': 1, 'hpylori': 1}
h plyori

h plyorie :
{'h pylori': 1, 'h pyloric': 1, 'h pylorie': 1, 'h plyori': 1, 'h plyorie': 1}
h plyorie

h pylori :
{'h pylori': 1, 'h pyloric': 1, 'h piloro': 1, 'h pylorie': 1, 'h plyori': 1, 'h plyorie': 1, 'pylori': 0, 'hpylori': 1, 'hp pyloric': 1, 'hel pylori': 1}
h pylori

h pyloric :
{'h pylori': 1, 'h pyloric': 1, 'h pylorie': 1, 'h

hernia repair

hernia repir :
{'hernia repairs': 1, 'hernia repar': 1, 'hurnia repair': 0, 'hernia repair': 2, 'hernia repir': 1}
hernia repair

hernia sgx :
{'hernia sx': 1, 'hernia sgx': 1}
hernia sgx

hernia surg :
{'hernia surg': 1}
hernia surg

hernia surgery :
{'hernia surgery': 1}
hernia surgery

hernia surgeryx2 :
{'hernia surgery': 1, 'hernia surgeryx2': 1}
hernia surgeryx2

hernia sx :
{'hernia on': 1, 'hernia sx': 1, 'hernia at': 1, 'hernia in': 1, 'hernias': 0, 'hernia to': 1, 'hernia sgx': 1}
hernia sx

hernia to :
{'hernia on': 1, 'hernia sx': 1, 'hernia at': 1, 'hernia in': 1, 'hernia to': 1}
hernia to

hernia umbilical :
{'hernia umbilical': 1}
hernia umbilical

hernia with :
{'hernia with': 1}
hernia with

herniated belly :
{'herniated belly': 1}
herniated belly

herniated diaphragm :
{'herniated diaphragm': 1}
herniated diaphragm

herniated dics :
{'herinated disc': 1, 'herniated disk': 1, 'herniated dics': 1, 'hernated disc': 1, 'herniates disc': 1, 'herniated discs'

homeopathic cough :
{'homeopathic cough': 1}
homeopathic cough

homeopathic for :
{'homeopathic for': 1}
homeopathic for

homicidal :
{'homicidal': 1}
homicidal

honey combed :
{'honey combed': 1}
honey combed

hyothyroidism :
{'hypothyroidsim': 5, 'hyothyroidism': 1, 'hypothyroidis': 0, 'hypthyroidism': 0, 'hopythyroidism': 0, 'hypothyridism': 0, 'hyptothyroidism': 0, 'hypthyrodism': 0, 'hypothyroidism': 0, 'hypothyroidisn': 0, 'hypothyroidsm': 0, 'hypothroidism': 0, 'hypothyrodism': 1, 'hypothyroidim': 0}
hypothyroidsim

hormonal acne :
{'hormonal acne': 1}
hormonal acne

hormonal imbalance :
{'hormonal imbalance': 1, 'hormone imbalance': 1}
hormonal imbalance

hormonal issues :
{'hormonal issues': 1}
hormonal issues

hormonal replacement :
{'hormonal replacement': 1, 'hormone replacement': 1}
hormonal replacement

hormone deficeincy :
{'hormone deficeincy': 1}
hormone deficeincy

hormone imbalance :
{'hormonal imbalance': 1, 'hormone imbalance': 1}
hormone imbalance

hormone problem

In [185]:
#'let'_clollected_words is made from the keys of a counter 
#so there should not be (and is not) a differnece in these numbers
len(h_collected_words), len(set(h_collected_words))

(939, 939)

In [186]:
h_collected_words[:10]

['h amputee',
 'h burn',
 'h foot',
 'h fracture',
 'h infection',
 'h injury',
 'h numbness',
 'h pain',
 'h piloro',
 'h plyori']

In [187]:
sorted(spell_dict.keys())[:10]

['h amputee',
 'h burn',
 'h foot',
 'h fracture',
 'h infection',
 'h injury',
 'h numbness',
 'h pain',
 'h piloro',
 'h plyori']

In [188]:
#this is my function for overwriting diagnoses with their correct spellings
for i in range(len(h_collected_words)):
    try:
        h_collected_words[i] = get_best_spelling(spell_dict[h_collected_words[i]])
    except:
        continue
    


In [189]:
h_collected_words[:20]

['h amputee',
 'h burn',
 'h foot',
 'h fracture',
 'h infection',
 'h injury',
 'h numbness',
 'h pain',
 'h piloro',
 'h plyori',
 'h plyorie',
 'h pylori',
 'h pyloric',
 'h pylorie',
 'h surgery',
 'h sx',
 'h years',
 'h1n1',
 'ha d',
 'hair loss']

In [190]:
len(set(h_collected_words))

939

### some work that I will not pursue on using only the first word of the diagnosis

In [1343]:
def first_word_counter(counter):
    first_words = []
    for word in counter.keys():
        first_word =  word.split()[0]
        first_words.append(first_word) 

    fw_counter = Counter(first_words)
    
    return fw_counter

In [1344]:
h_fw_counter = first_word_counter(new_h_counter)

In [1345]:
len(h_fw_counter)

313

In [1346]:
h_fw_counter.most_common()

[('heart', 61),
 ('high', 39),
 ('hip', 31),
 ('history', 27),
 ('htn', 20),
 ('hernia', 15),
 ('hysterectomy', 13),
 ('hearing', 11),
 ('hypertension', 10),
 ('hand', 9),
 ('hep', 9),
 ('hepatic', 9),
 ('heavy', 8),
 ('hemiplegia', 7),
 ('hiv', 7),
 ('head', 6),
 ('hemorrhoids', 6),
 ('herniated', 6),
 ('home', 6),
 ('headache', 5),
 ('heel', 5),
 ('hypertrophic', 5),
 ('healthy', 4),
 ('hemangioma', 4),
 ('hereditary', 4),
 ('herpes', 4),
 ('hypo', 4),
 ('holter', 4),
 ('hormonal', 4),
 ('hormone', 4),
 ('hyperactive', 4),
 ('hyperplasia', 4),
 ('h', 3),
 ('hear', 3),
 ('hallucinations', 3),
 ('hemodialysis', 3),
 ('hematuria', 3),
 ('hepatitis', 3),
 ('hgb', 3),
 ('hodgkins', 3),
 ('hospitalized', 3),
 ('htnn', 3),
 ('huntingtons', 3),
 ('hydrocele', 3),
 ('hydrocephalus', 3),
 ('hyperemesis', 3),
 ('hypoglycemia', 3),
 ('h-pylori', 2),
 ('hamstring', 2),
 ('he', 2),
 ('healed', 2),
 ('hematoma', 2),
 ('hemicolectomy', 2),
 ('hemophilia', 2),
 ('hemorrhagic', 2),
 ('her', 2),
 ('her

now we are going to collapse the diagnoses to only care about the first word...

In [132]:
h_first_words = []
for word in h_counter.keys():
    first_word =  word.split()[0]
    h_first_words.append(first_word) 
        
h_fw_counter = Counter(h_first_words)

In [133]:
#much shorter but a lot of information is lost
len(h_fw_counter)

316

In [134]:
h_fw_counter.most_common()[100:120]

[('hemorroids', 1),
 ('hairline', 1),
 ('half', 1),
 ('haliperdol', 1),
 ('hamilton', 1),
 ('harada', 1),
 ('haringtons', 1),
 ('hartmann', 1),
 ('hay', 1),
 ('hb', 1),
 ('hbd', 1),
 ('hbh', 1),
 ('hbv', 1),
 ('hcl', 1),
 ('hctz', 1),
 ('hd', 1),
 ('hdhd', 1),
 ('headinjury', 1),
 ('heaing', 1),
 ('heal', 1)]

In [135]:
sorted(h_counter.most_common()[100:120])

[('hctz', 1),
 ('hd tuethusat', 1),
 ('hdhd', 1),
 ('he a', 1),
 ('he does', 1),
 ('head ache', 1),
 ('head concusion', 1),
 ('head fracture', 1),
 ('head injjury', 1),
 ('head injuries', 1),
 ('head injury', 1),
 ('head neck', 1),
 ('head pain', 1),
 ('head surgery', 1),
 ('head sx', 1),
 ('head trauma', 1),
 ('head tumor', 1),
 ('head tumour', 1),
 ('headache concussion', 1),
 ('heart faillure', 1)]

In [346]:
#what happens if you make all the items with the same first word are considered the same?
first_words = []
for word in c_counter.keys():
    first_word =  word.split()[0]
    first_words.append(first_word) 

In [347]:
first_words[20:30]

['c-palate',
 'c-palate',
 'c-palate',
 'c-pap',
 'c-pap',
 'c-sction',
 'c-sec',
 'c-sect',
 'c-sect',
 'c-sect']

In [348]:
#this is promising.  decreases the number of diagnoses in half
fw_counter = Counter(first_words); len(fw_counter)

643

### some regex experimentation

In [238]:
hp = poss_words('h pylori')

In [239]:
hp

['h pylori',
 'h pyloric',
 'h piloro',
 'h pylorie',
 'h plyori',
 'h plyorie',
 'pylori',
 'hpylori',
 'hp pyloric',
 'hel pylori']

In [242]:
for h in hp:
    print (re.match('\b[a-z]\s[a-z]*', hp))

TypeError: expected string or bytes-like object

In [247]:
s = re.sub(r"([a-z])\s([a-z])", r"\1-\2", hp[0]) 

In [248]:
s

'h-pylori'

In [251]:
s = re.sub(r"([a-z])\s([a-z])", r"\1-\2", 'a fib'); s

'a-fib'

In [265]:
s = re.sub(r"([a-z])\s([a-z])", r"\1-\2", 'h pylori'); s

'h-pylori'

In [275]:
s = re.sub(r"(\b[a-z]\b)\s([a-z])", r"\1-\2", 'h pr'); s

'h-pr'